<a href="https://colab.research.google.com/github/your-repo/cscg_torch/blob/main/examples/colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSCG PyTorch Demo - Google Colab

This notebook demonstrates the GPU-optimized CSCG PyTorch implementation in Google Colab.

## Setup

In [ ]:
# Install dependencies
!pip install torch>=2.1.0 numpy>=1.24 matplotlib>=3.7 tqdm>=4.65 scipy>=1.10

# Clone repository (replace with actual repo URL)
!git clone https://github.com/your-repo/cscg_torch.git

# Add to Python path
import sys
sys.path.append('/content/cscg_torch')

: 

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("No GPU available - using CPU")

## Basic Usage

In [ ]:
from cscg_torch import CHMM_torch

# Generate synthetic data
np.random.seed(42)
torch.manual_seed(42)

# Define model structure
n_clones = torch.tensor([3, 4, 2, 3], dtype=torch.int64)  # clones per observation
sequence_length = 200

# Generate random sequences
x = torch.randint(0, len(n_clones), (sequence_length,), dtype=torch.int64)
a = torch.randint(0, 3, (sequence_length-1,), dtype=torch.int64)  # 3 actions

print(f"Observation sequence length: {len(x)}")
print(f"Action sequence length: {len(a)}")
print(f"Number of observation types: {len(n_clones)}")
print(f"Total clone states: {n_clones.sum()}")

In [ ]:
# Create CHMM model
model = CHMM_torch(
    n_clones=n_clones,
    x=x,
    a=a,
    pseudocount=1e-6,
    dtype=torch.float32,
    seed=42
)

print("Model created successfully!")
print(f"Model device: {model.device}")
print(f"Transition matrix shape: {model.T.shape}")

## Training

In [ ]:
# EM Training
print("Starting EM training...")
convergence_em = model.learn_em_T(x, a, n_iter=50, term_early=True)

print(f"\nEM Training completed!")
print(f"Final BPS: {convergence_em[-1]:.4f}")
print(f"Converged in {len(convergence_em)} iterations")

In [ ]:
# Plot convergence
plt.figure(figsize=(10, 6))
plt.plot(convergence_em, 'b-', linewidth=2, label='EM Training')
plt.xlabel('Iteration')
plt.ylabel('Bits per Step (BPS)')
plt.title('CHMM Training Convergence')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

print(f"Improvement: {convergence_em[0] - convergence_em[-1]:.4f} BPS")

## Inference and Decoding

In [ ]:
# Compute likelihood
bps_total = model.bps(x, a, reduce=True)
bps_per_step = model.bps(x, a, reduce=False)

print(f"Total BPS: {bps_total:.4f}")
print(f"Average BPS: {bps_per_step.mean():.4f}")
print(f"BPS std: {bps_per_step.std():.4f}")

In [ ]:
# MAP decoding
map_likelihood, map_states = model.decode(x, a)

print(f"MAP log-likelihood: {-map_likelihood:.4f}")
print(f"MAP states shape: {map_states.shape}")
print(f"First 10 MAP states: {map_states[:10]}")

# Viterbi likelihood
viterbi_bps = model.bpsV(x, a)
print(f"Viterbi BPS: {viterbi_bps:.4f}")

## Emission Learning

In [ ]:
# Learn emission matrix
print("Learning emission matrix...")
convergence_e, E = model.learn_em_E(x, a, n_iter=30)

print(f"Emission learning completed!")
print(f"Final emission BPS: {convergence_e[-1]:.4f}")
print(f"Emission matrix shape: {E.shape}")

In [ ]:
# Inference with learned emissions
bps_with_emissions = model.bpsE(E, x, a)
map_lik_e, map_states_e = model.decodeE(E, x, a)

print(f"BPS with emissions: {bps_with_emissions:.4f}")
print(f"MAP likelihood with emissions: {-map_lik_e:.4f}")

# Compare emission matrix
plt.figure(figsize=(12, 4))
plt.imshow(E.cpu().numpy(), aspect='auto', cmap='viridis')
plt.colorbar(label='Emission Probability')
plt.xlabel('Observation Type')
plt.ylabel('Clone State')
plt.title('Learned Emission Matrix')
plt.tight_layout()
plt.show()

## Sequence Generation

In [ ]:
# Generate sequences from learned model
sample_length = 50
sample_x, sample_a = model.sample(sample_length)

print(f"Generated sequence length: {len(sample_x)}")
print(f"Sample observations: {sample_x[:20]}")
print(f"Sample actions: {sample_a[:20]}")

# Conditional generation
conditional_seq = model.sample_sym(sym=0, length=20)
print(f"Conditional sequence starting with 0: {conditional_seq}")

## Performance Comparison

In [ ]:
import time

# Benchmark GPU vs CPU (if both available)
if torch.cuda.is_available():
    # GPU timing
    model_gpu = CHMM_torch(n_clones, x, a, dtype=torch.float32)
    
    torch.cuda.synchronize()
    start_time = time.time()
    _ = model_gpu.learn_em_T(x, a, n_iter=10)
    torch.cuda.synchronize()
    gpu_time = time.time() - start_time
    
    print(f"GPU training time (10 iterations): {gpu_time:.2f}s")
    
    # Memory usage
    if torch.cuda.is_available():
        memory_used = torch.cuda.max_memory_allocated() / 1e6
        print(f"Peak GPU memory usage: {memory_used:.1f} MB")
else:
    print("GPU not available for benchmarking")

## Summary

This notebook demonstrated:

1. **Setup**: Installing dependencies and checking GPU availability
2. **Model Creation**: Building CHMM with clone states
3. **Training**: EM algorithm for transition learning
4. **Inference**: Likelihood computation and MAP decoding
5. **Emissions**: Learning and using emission matrices
6. **Generation**: Sampling sequences from trained models
7. **Performance**: GPU acceleration benefits

The CSCG PyTorch implementation provides efficient GPU-accelerated training and inference for Hidden Markov Models with compositional structure.